In [70]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from kiteconnect import KiteConnect
from kiteconnect import KiteTicker
from datetime import datetime,timedelta
import pandas as pd
import numpy as np
import time
import requests


import warnings
warnings.filterwarnings("ignore")

def access_token(api_secret, request_token, kite):
    user = pd.DataFrame()
    try:
        user = kite.generate_session(request_token, api_secret)
       
        print("Access token ->" + user["access_token"])
        acc_key=user["access_token"]
   
    except Exception as e:
        print("Authentication failed", str(e))
        raise

    print(user["user_name"], "has successfully signed in.")
    return user



acc_key=0

while(acc_key==0):
    user_name = "RK2267"
    password = "Shan@#2021"
#     totp_key = "RGHL2AKPKBTQUPR35WFKWDTIO5NJEALQ"
    options = Options()
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument('--disble-dev-shm-usage')
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get("https://kite.trade/connect/login?api_key=pv2830q1vbrhu1eu")
    time.sleep(1)
    element = driver.find_element_by_id("userid")
    element.send_keys(user_name)
    element = driver.find_element_by_id("password")
    element.send_keys(password)
    element.send_keys(Keys.RETURN)
    time.sleep(15)
#     outh = pyotp.TOTP(totp_key)
#     element = driver.find_element_by_id('externaltotp')
#     element.send_keys(outh.now())
#     element.send_keys(Keys.RETURN)
#     time.sleep(1)
    # element.close()
    print(driver.current_url)
    url = driver.current_url.split("request_token=")[1].split("&")[0]
    print(url, "new url")
    driver.quit()
    my_api_key = "pv2830q1vbrhu1eu"
    kite = KiteConnect(api_key=my_api_key)
    req_key = url
    user_details = access_token(
        "gz426yatawim9xruj2p6e51c40k44r7n", req_key, kite)
    acc_key = user_details["access_token"]
#     print("Access token ->" + acc_key)
    kite.set_access_token(acc_key)

https://kite.trade/?type=login&status=success&request_token=vw8YY61HYJyRVeYQGjN2ttRrOLnYx09t&action=login
vw8YY61HYJyRVeYQGjN2ttRrOLnYx09t new url
Access token ->Bnmmlp2PbTwgwU2CMkPGz8hj5aYb0xhW
Kakal Krishna Rao has successfully signed in.


In [156]:
from datetime import *

def history_data(ticker_val):
    data1=['' for i in range(1)]
    start_date=(datetime.now() - timedelta(days=100))
    curr_date=datetime.now()
    if 9<=curr_date.hour<16:
        new_date=(datetime.now() - timedelta(days=1))
    else:
        new_date=(datetime.now())
    
    startd=pd.to_datetime(str(start_date)[:10]+" "+'00:00:00.0')
    endd=pd.to_datetime(str(new_date)[:10]+' '+'15:30:00.0')
    
    url1="https://api.kite.trade/instruments/historical/"+str(ticker_val)+"/day?from="+str(startd)+"&to="+str(endd)
    #print(url1)
    HEADERS = {"X-Kite-Version": "3", "Authorization":"token pv2830q1vbrhu1eu:"+acc_key}
    res1 = requests.get(url1, headers=HEADERS)
    i=0
    data1[i] = res1.json()
    data1[i] = data1[i]["data"]["candles"]
    
    data1[i]=pd.DataFrame(data1[i])
    data1[i] = data1[i].rename(columns={0: 'Time', 1: 'Open', 2:'High', 3:'Low', 4:'Close', 5:'Volume'})
    pd.set_option("display.max_rows", None, "display.max_columns", None)
    print('History_data {} collected .'.format(ticker_val))
    return data1

In [157]:
from termcolor import colored
def crossover(dfx,tkname):
    def macd(df,m1,m2,s1):
        exp1 = df.Close.ewm(span=m1, adjust=False).mean()
        exp2 = df.Close.ewm(span=m2, adjust=False).mean()
        macd2 = exp1-exp2
        exp3 = macd2.ewm(span=s1, adjust=False).mean()
        return macd2,exp3
    def change(s):
        return s[:10]

    dfx['Time']=dfx.Time.apply(change)
    macd,sig=macd(dfx,12,26,9)
    dfx['macd']=macd
    dfx['sig']=sig
    last10=dfx.loc[len(dfx)-11:]
    buy=0
    sell=0
    nocross=0
    print('CALCULATING 10DAYS CROSSOVER OF',tkname)
    last10.reset_index(inplace=True,drop=True)
    print()
    for i in range(1,len(last10)):
        print('Date->',last10['Time'].iloc[i],'MACD ->',last10['macd'].iloc[i],'SIG->',last10['sig'].iloc[i])
    print()
    for i in range(1,len(last10)):
        if last10['macd'].iloc[i]>last10['sig'].iloc[i] and last10['macd'].iloc[i-1]<last10['sig'].iloc[i-1]:
            print(colored('LONG ENTRY on {}'.format(last10['Time'].iloc[i]),'green'))
            buy+=1
        elif last10['macd'].iloc[i]<last10['sig'].iloc[i] and last10['macd'].iloc[i-1]>last10['sig'].iloc[i-1]:
            print(colored('SHORT ENTRY on {}'.format(last10['Time'].iloc[i]),'red'))
            sell+=1
        else:
            print(colored('No crossover on {}'.format(last10['Time'].iloc[i]),'blue'))
            nocross+=1

    print()
    print('--------- SUMMARY ---------------')
    print('Start date ->',last10['Time'].iloc[1])
    print('No of Long Entry :',buy)
    print('No of Short Entry :',sell)
    print('End date ->',last10['Time'].iloc[-1])
    print('----------------------------------')
    print()
    
    
    
dicct={67329: 'ATUL', 5436929: 'AUBANK', 70401: 'AUROPHARMA', 1510401: 'AXISBANK', 4267265: 'BAJAJ-AUTO', 4268801: 'BAJAJFINSV', 81153: 'BAJFINANCE', 85761: 'BALKRISIND', 87297: 'BALRAMCHIN', 579329: 'BANDHANBNK', 1195009: 'BANKBARODA', 94977: 'BATAINDIA', 98049: 'BEL', 103425: 'BERGEPAINT', 108033: 'BHARATFORG', 2714625: 'BHARTIARTL', 112129: 'BHEL', 2911489: 'BIOCON', 558337: 'BOSCHLTD', 134657: 'BPCL'}
tokens=[67329, 5436929, 70401, 1510401, 4267265, 4268801, 81153, 85761, 87297, 579329, 1195009, 94977, 98049, 103425, 108033, 2714625, 112129, 2911489, 558337, 134657]

for tk in tokens:
    df=history_data(tk)[0]
    crossover(df,dicct[tk])

History_data 67329 collected .
CALCULATING 10DAYS CROSSOVER OF ATUL

Date-> 2022-09-30 MACD -> -46.376637156043216 SIG-> 19.715573790925205
Date-> 2022-10-03 MACD -> -57.910231024470704 SIG-> 4.190412827846025
Date-> 2022-10-04 MACD -> -56.71791589546774 SIG-> -7.9912529168167294
Date-> 2022-10-06 MACD -> -58.15678106909036 SIG-> -18.024358547271458
Date-> 2022-10-07 MACD -> -66.70225342940466 SIG-> -27.7599375236981
Date-> 2022-10-10 MACD -> -83.2030964076639 SIG-> -38.84856930049126
Date-> 2022-10-11 MACD -> -102.8490207193172 SIG-> -51.64865958425645
Date-> 2022-10-12 MACD -> -115.10267628292968 SIG-> -64.3394629239911
Date-> 2022-10-13 MACD -> -118.07060954464396 SIG-> -75.08569224812169
Date-> 2022-10-14 MACD -> -121.2082141280971 SIG-> -84.31019662411677

No crossover on 2022-09-30
No crossover on 2022-10-03
No crossover on 2022-10-04
No crossover on 2022-10-06
No crossover on 2022-10-07
No crossover on 2022-10-10
No crossover on 2022-10-11
No crossover on 2022-10-12
No crossover

History_data 85761 collected .
CALCULATING 10DAYS CROSSOVER OF BALKRISIND

Date-> 2022-09-30 MACD -> -62.43619977143817 SIG-> -63.56184385032394
Date-> 2022-10-03 MACD -> -60.530860704329825 SIG-> -62.95564722112512
Date-> 2022-10-04 MACD -> -54.66678182606165 SIG-> -61.297874142112434
Date-> 2022-10-06 MACD -> -49.28989060432946 SIG-> -58.89627743455584
Date-> 2022-10-07 MACD -> -44.7508460220115 SIG-> -56.06719115204697
Date-> 2022-10-10 MACD -> -41.79346526923223 SIG-> -53.21244597548402
Date-> 2022-10-11 MACD -> -38.46168872196381 SIG-> -50.26229452477999
Date-> 2022-10-12 MACD -> -32.79249981772841 SIG-> -46.76833558336968
Date-> 2022-10-13 MACD -> -29.436952957512858 SIG-> -43.30205905819832
Date-> 2022-10-14 MACD -> -27.409821338169877 SIG-> -40.12361151419263

LONG ENTRY on 2022-09-30
No crossover on 2022-10-03
No crossover on 2022-10-04
No crossover on 2022-10-06
No crossover on 2022-10-07
No crossover on 2022-10-10
No crossover on 2022-10-11
No crossover on 2022-10-12
No cros

History_data 108033 collected .
CALCULATING 10DAYS CROSSOVER OF BHARATFORG

Date-> 2022-09-30 MACD -> -11.090619767928388 SIG-> -0.1934146639993033
Date-> 2022-10-03 MACD -> -13.060047463262208 SIG-> -2.7667412238518847
Date-> 2022-10-04 MACD -> -13.245666911591798 SIG-> -4.862526361399867
Date-> 2022-10-06 MACD -> -8.701116144196703 SIG-> -5.630244317959235
Date-> 2022-10-07 MACD -> -4.766198977215481 SIG-> -5.457435249810485
Date-> 2022-10-10 MACD -> -0.7674325475039723 SIG-> -4.5194347093491825
Date-> 2022-10-11 MACD -> 2.0551614649195926 SIG-> -3.2045154744954276
Date-> 2022-10-12 MACD -> 3.4813515149389787 SIG-> -1.8673420766085465
Date-> 2022-10-13 MACD -> 3.2547898559520263 SIG-> -0.842915690096432
Date-> 2022-10-14 MACD -> 3.3353493555382556 SIG-> -0.0072626809694944505

No crossover on 2022-09-30
No crossover on 2022-10-03
No crossover on 2022-10-04
No crossover on 2022-10-06
LONG ENTRY on 2022-10-07
No crossover on 2022-10-10
No crossover on 2022-10-11
No crossover on 2022-10